### Code to time models against sample feeder images
set up code to load all images into memory  
runs all images through a model and times model at the task  
runs multiple models for comparision
loaded in a 2cpu 2gb compute resource for timing differences

In [16]:
import tensorflow as tf
from tensorflow import keras
from PIL import Image
import numpy as np
import os
import time

from tensorflow.keras.applications import MobileNetV2, EfficientNetB0

In [2]:
image_dir = 'home/pi/batch_test'  # relative to birdclassifier dir in jupyter nb

In [3]:
# get the classes, number space text with a line end 
classes_dict = {}
with open('NAbirdsClasses.txt', 'r') as f:
    for line in f:
        line = line.strip()
        if line:  # Ignore empty lines
            parts = line.split(' ', 1)
            if len(parts) == 2 and parts[0].isdigit():
                key = int(parts[0])
                value = parts[1]
                classes_dict[key] = value
# print(classes_dict)

In [4]:
def list_model_h5_files(prefix='Experiment5'):
    matching_files = []
    for filename in os.listdir():  # current dir
        if filename.startswith(prefix) and filename.endswith('.h5'):
            matching_files.append(filename)
    return matching_files

In [5]:
def load_model(model_path):
    # load the model and get input size
    model = keras.models.load_model(model_path)
    # print(model.input_shape)  
    input_shape = model.input_shape[1:3]  # height, width, channel in format (None, 224, 224, 3)
    return model, input_shape

In [6]:
def preprocess_image(image_path, target_size):
    img = Image.open(image_path).resize(target_size)
    img_array = np.array(img)
    # img_array = img_array / 255.0  # done in model? 
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension, we are doing this one at a time
    return img_array

In [7]:
def batch_predict_model(image_files, image_dir, model, input_shape, classes_dict):
    # includes load time, but that needs to happen for resizing
    images_count = len(image_files)
    preprocessed_images = []
    for ii, image_file in enumerate(image_files):
        image_path = os.path.join(image_dir, image_file)
        try:
            preprocessed_images.append(preprocess_image(image_path, input_shape))
        except Exception as e:
            print(f'error processing image {image_file}: {e}')
            
    start_time = time.time()
    for ii, pimage in enumerate(preprocessed_images):
        try:
            prediction = model.predict(pimage)
            predicted_class_index = np.argmax(prediction)
            confidence = prediction[0][predicted_class_index] * 100
            # print(f'image {ii} of {images_count}: {image_file}, Prediction: {predicted_class_index}:{classes_dict[predicted_class_index]}, Confidence: {confidence:.2f}%')

        except Exception as e:
            print(f'error making prediction {image_file}: {e}')

    stop_time = time.time()
    total_run_time = stop_time - start_time
    print(f'\n run time: {total_run_time:.2f} seconds')
    return

In [14]:
# Load the MobileNetV2 model with default weights and run against images, it will be wrong but timings will be valid
base_model = MobileNetV2()

image_files = [f for f in os.listdir(image_dir) if os.path.isfile(os.path.join(image_dir, f))]
input_shape = model.input_shape[1:3]  # height, width, channel in format (None, 224, 224, 3)
batch_predict_model(image_files, image_dir, model, input_shape, classes_dict)


1/1 [==============================] - 0s 123ms/step

 run time: 28.21 seconds


In [17]:
# Load the EfficientNetB0 model with default weights and run against images, it will be predict wrong but timings will be valid
base_model = EfficientNetB0()

image_files = [f for f in os.listdir(image_dir) if os.path.isfile(os.path.join(image_dir, f))]
input_shape = model.input_shape[1:3]  # height, width, channel in format (None, 224, 224, 3)
batch_predict_model(image_files, image_dir, model, input_shape, classes_dict)


1/1 [==============================] - 0s 94ms/step

 run time: 26.97 seconds


In [18]:
# code to run and time actual models
# model_files = []
# # model_files = list_model_h5_files(prefix='Experiment')
# # model_files.append('coral.ai.mobilenet_v2_1.0_224_inat_bird_quant.tflite')  # add directories manually
# # model_files.append('Experiment5:MobileNetV2MobileNetV2.h5')  # add directories manually
# model_files.append('efficientnetb0_3.h5')  # add directories manually
# image_files = [f for f in os.listdir(image_dir) if os.path.isfile(os.path.join(image_dir, f))]

# for model_filename in model_files:
#     print('*************************')
#     print(f'model: {model_filename}')
#     try:
#         model, input_shape = load_model(model_filename)
#         batch_predict_model(image_files, image_dir, model, input_shape, classes_dict)
#         print('')
#     except Exception as e:
#         print (f'error with model {model_filename} as {e}')

In [9]:
print(len(image_files))  # number of images

160
